In [0]:
#configuring with Custom access token
configs = {"fs.azure.account.auth.type": "CustomAccessToken",
           "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}


#Mounting our storage to databricks
dbutils.fs.mount(
  source = "abfss://bronze@endtoendstoresyn.dfs.core.windows.net/",
  mount_point = "/mnt/bronze",
  extra_configs = configs)

True

# Load Raw Data

In [0]:
dbutils.fs.ls("/mnt/bronze")

[FileInfo(path='dbfs:/mnt/bronze/usvideos.csv', name='usvideos.csv', size=333337, modificationTime=1729217481000)]

In [0]:
from pyspark.sql.functions import col, regexp_replace, split, udf, when,  to_timestamp
from pyspark.sql.types import FloatType, IntegerType, StringType, DateType,TimestampType


# Load the dataset from the bronze container
bronze_df = spark.read.csv('/mnt/bronze/usvideos.csv', header=True, inferSchema=True)

#Display the raw data from bronze container
display(bronze_df)

VIDEO_ID,TITLE,PUBLISHEDAT,CHANNELID,CHANNELTITLE,CATEGORYID,TRENDING_DATE,TAGS,VIEW_COUNT,LIKES,DISLIKES,COMMENT_COUNT,THUMBNAIL_LINK,DESCRIPTION
U68MJz9DrI4,"Gucci Mane, Bruno Mars, Kodak Black - Wake Up in The Sky [Official Music Video]",2018-10-31 14:17:10.000,UCSugZEYrWbzqIWGD195V-YA,OfficialGucciMane,10,18.01.11,officialguccimane|gucci mane|gucci|mane|atlantic records|atlantic|guwop|radric davis|hip hop|rap|i get the bag|both|curve|met gala|solitaire|real rich|el gato|mr. davis|droptopwop|atlanta|DN6WWu_KEQs|UdvpUjCmLC4|Wake Up In The Sky|Bruno Mars|Kodak Black|Music Video|Official Music Video|mjaayCARwro|zcWqllTtns4|gucci mane bruno mars kodak black|smoke til I'm high|drink til I'm drunk|You cant tell me I aint fly|I know Im super fly|wuits|gucci bruno kodak,673533,91342,1480,8204,https://i.ytimg.com/vi/U68MJz9DrI4/default.jpg,"The official music video for Gucci Mane, Bruno Mars, and Kodak Black’s “Wake Up In The Sky” – available now!Stream/Download - https://guccimane.lnk.to/WakeUpInTheSkyAYDirector: Bruno Mars & Florent DechardProduced By Jeremy SullivanSubscribe for more official content from Gucci Mane:https://Atlantic.lnk.to/GMsubscribeFollow Gucci Manehttp://GucciManeOnline.comhttp://Twitter.com/Gucci1017http://Facebook.com/GucciManehttp://Instagram.com/LaFlare1017Follow Bruno Marshttp://www.brunomars.comhttp://www.instagram.com/brunomarshttp://www.twitter.com/brunomarshttp://www.facebook.com/brunomarsFollow Kodak Blackhttps://officialkodakblack.com/https://twitter.com/KodakBlack1khttps://facebook.com/TheRealKodakBlackhttps://instagram.com/kodakblackhttps://soundcloud.com/kodak-blackThe official YouTube channel of Atlantic Records artist Gucci Mane. Subscribe for the latest music videos, performances, and more.#gucci #kodak #bruno #guccimane #brunomars #kodakblack #WakeUpInTheSky #WUITS #MusicVideo #AtlanticRecords #atlantic"
shWNgr0ifrs,Kevin Gates - Great Man [Official Music Video],2018-10-31 14:00:08.000,UCj2GTFekdV3EUsTVN8oaEqA,kevingatesTV,10,18.01.11,kevin gates|kevin|gates|atlantic|atlantic records|how we livin|2 phones|really really|by any means 2|Great Man|Luca Brasi 3|Luca Brasi|Luca Brasi 2|Adding Up|Me Too|Money Long|Music Video|Official Music Video|BWA|Bread Winners|p1DP9BSIL-g|4i8tXcTXs5M,207289,18647,256,1707,https://i.ytimg.com/vi/shWNgr0ifrs/default.jpg,"Kevin Gates - Great ManStream/Download - https://kevingates.lnk.to/LucaBrasi3Subscribe for more official content from Kevin Gates:https://kevingates.lnk.to/SubscribeFollow GatesTwitter: http://twitter.com/kevin_gatesInstagram: http://instagram.com/iamkevingatesFacebook: https://facebook.com/kvngatesSoundcloud: https://soundcloud.com/kevingatesWebsite: http://www.kvngates.comThe official YouTube channel of Atlantic Records artist Kevin Gates. Subscribe for the latest music videos, performances, and more."
JIFMN986m8s,Worst Halloween Candy Taste Test (Day 3),2018-10-31 10:00:03.000,UC4PooiX37Pld1T8J5SYT-SQ,Good Mythical Morning,24,18.01.11,gmm|good mythical morning|rhettandlink|rhett and link|mythical morning|mythical|rhett|link|season 14|gmm candy|gmm candy tournament|good mythical morning candy|good mythical morning candy tournament|halloween|candy|halloween candy|worst halloween candy|taste test|taste|test|gmm tournament|good mythical morning tournament|clash of the crappy candies|munch madness|gmm munch madness|rhett and link candy|Worst Halloween Candy Taste Test (Day 3),1054156,39129,1236,10153,https://i.ytimg.com/vi/JIFMN986m8s/default.jpg,"We've arrived at the finals! Which of our Ehh Eight will be crowned the crappiest candy? Tune into LTAT on Saturday to see how the Mythical Crew's brackets fared! GMM #1412Watch today's GMMore: https://youtu.be/L6SmOn2dRhkWant more GMM? Watch this season from the start: http://bit.ly/GMM_S14Pick up official GMM and Mythical merch at https://mythical.storeand https://www.amazon.com/mythicalDon’t miss our weekly podcast, Ear Biscuits: https://applepodcasts.com/earbiscuitsFollow Rhett & Link: Instagram: https

# Make Transformations to data

In [0]:
dbutils.fs.mount(
  source = "abfss://silver@endtoendstoresyn.dfs.core.windows.net/",
  mount_point = "/mnt/silver",
  extra_configs = configs)


True

In [0]:
# Transformations
# 1. Drop unnecessary columns
silver_df = bronze_df.drop('TRENDING_DATE')

# 2. Capitalize first letter in each column only, but keep underscores
silver_df = silver_df.select(
    col('VIDEO_ID').alias('Video_id'),
    col('TITLE').alias('Title'),
    col('PUBLISHEDAT').alias('Published_at'),
    col('CHANNELID').alias('Channel_id'),
    col('CHANNELTITLE').alias('Channel_title'),
    col('CATEGORYID').alias('Category_id'),
    col('TAGS').alias('Tags'),
    col('VIEW_COUNT').alias('View_count'),
    col('LIKES').alias('Likes'),
    col('DISLIKES').alias('Dislikes'),
    col('COMMENT_COUNT').alias('Comment_count'),
    col('THUMBNAIL_LINK').alias('Thumbnail_link'),
    col('DESCRIPTION').alias('Description')
)

# 3. Remove invalid characters in all column names
for col_name in silver_df.columns:
    new_col_name = col_name.replace(" ", "_").replace("-", "_")
    silver_df = silver_df.withColumnRenamed(col_name, new_col_name)

# 4. Cast columns to correct type
silver_df = silver_df.withColumn('View_count', col('View_count').cast(IntegerType())
).withColumn('Comment_count', col('Comment_count').cast(IntegerType())
).withColumn('Category_id', col('Category_id').cast(IntegerType())
).withColumn('Likes', col('Likes').cast(IntegerType())
).withColumn('Dislikes', col('Dislikes').cast(IntegerType())
).withColumn('Published_at', col('Published_at').cast(DateType()))

# 5. Tag Standardization; Keep only main tag
silver_df = silver_df.withColumn("Tags", split(col("Tags"), "\\|").getItem(0))

# 6. Remove invalid rows with missing value
silver_df = silver_df.na.drop(subset=["Category_id"])

# Display the updated dataframe
display(silver_df)

Video_id,Title,Published_at,Channel_id,Channel_title,Category_id,Tags,View_count,Likes,Dislikes,Comment_count,Thumbnail_link,Description
U68MJz9DrI4,"Gucci Mane, Bruno Mars, Kodak Black - Wake Up in The Sky [Official Music Video]",2018-10-31,UCSugZEYrWbzqIWGD195V-YA,OfficialGucciMane,10,officialguccimane,673533,91342,1480,8204,https://i.ytimg.com/vi/U68MJz9DrI4/default.jpg,"The official music video for Gucci Mane, Bruno Mars, and Kodak Black’s “Wake Up In The Sky” – available now!Stream/Download - https://guccimane.lnk.to/WakeUpInTheSkyAYDirector: Bruno Mars & Florent DechardProduced By Jeremy SullivanSubscribe for more official content from Gucci Mane:https://Atlantic.lnk.to/GMsubscribeFollow Gucci Manehttp://GucciManeOnline.comhttp://Twitter.com/Gucci1017http://Facebook.com/GucciManehttp://Instagram.com/LaFlare1017Follow Bruno Marshttp://www.brunomars.comhttp://www.instagram.com/brunomarshttp://www.twitter.com/brunomarshttp://www.facebook.com/brunomarsFollow Kodak Blackhttps://officialkodakblack.com/https://twitter.com/KodakBlack1khttps://facebook.com/TheRealKodakBlackhttps://instagram.com/kodakblackhttps://soundcloud.com/kodak-blackThe official YouTube channel of Atlantic Records artist Gucci Mane. Subscribe for the latest music videos, performances, and more.#gucci #kodak #bruno #guccimane #brunomars #kodakblack #WakeUpInTheSky #WUITS #MusicVideo #AtlanticRecords #atlantic"
shWNgr0ifrs,Kevin Gates - Great Man [Official Music Video],2018-10-31,UCj2GTFekdV3EUsTVN8oaEqA,kevingatesTV,10,kevin gates,207289,18647,256,1707,https://i.ytimg.com/vi/shWNgr0ifrs/default.jpg,"Kevin Gates - Great ManStream/Download - https://kevingates.lnk.to/LucaBrasi3Subscribe for more official content from Kevin Gates:https://kevingates.lnk.to/SubscribeFollow GatesTwitter: http://twitter.com/kevin_gatesInstagram: http://instagram.com/iamkevingatesFacebook: https://facebook.com/kvngatesSoundcloud: https://soundcloud.com/kevingatesWebsite: http://www.kvngates.comThe official YouTube channel of Atlantic Records artist Kevin Gates. Subscribe for the latest music videos, performances, and more."
JIFMN986m8s,Worst Halloween Candy Taste Test (Day 3),2018-10-31,UC4PooiX37Pld1T8J5SYT-SQ,Good Mythical Morning,24,gmm,1054156,39129,1236,10153,https://i.ytimg.com/vi/JIFMN986m8s/default.jpg,"We've arrived at the finals! Which of our Ehh Eight will be crowned the crappiest candy? Tune into LTAT on Saturday to see how the Mythical Crew's brackets fared! GMM #1412Watch today's GMMore: https://youtu.be/L6SmOn2dRhkWant more GMM? Watch this season from the start: http://bit.ly/GMM_S14Pick up official GMM and Mythical merch at https://mythical.storeand https://www.amazon.com/mythicalDon’t miss our weekly podcast, Ear Biscuits: https://applepodcasts.com/earbiscuitsFollow Rhett & Link: Instagram: https://instagram.com/rhettandlinkFacebook: https://facebook.com/rhettandlinkTwitter: https://twitter.com/rhettandlinkWebsite: https://mythical.com/Check Out Our Other Mythical Channels:Good Mythical MORE: https://youtube.com/goodmythicalmoreRhett & Link: https://youtube.com/rhettandlinkMythical: https://youtube.com/thisismythicalWant to send us something? https://mythical.com/contactSubmit your Wheel of Mythicality intro video here: https://bit.ly/GMMWheelIntroIntro Animation by Digital Twigs: https://www.digitaltwigs.comIntro & Outro Music by Jeff Zeigler & Sarah Schimeneck https://www.jeffzeigler.comSupplemental Music from Extreme Production Music: https://www.extrememusic.com/Mic: ‘The Mouse’ by Blue Microphones https://www.bluemic.com/mouse/"
0iy3HPxBFQY,James Corden & Ariana Grande Visit an Escape Room,2018-10-31,UCJ0uqCI0Vqr2Rrt1HseGirg,The Late Late Show with James Corden,24,The Late Late Show,896693,49252,289,2247,https://i.ytimg.com/vi/0iy3HPxBFQY/default.jpg,"Since Ariana Grande loves Halloween and being scared, James takes her to a haunted escape room and they both are terrified while trying to get out. Thank you to 60OUT Escape Rooms. Learn more about visiting one: ht

In [0]:
# Save silver_df to silver directory in delimited text format in a single file
# Define the output path for the "silver" directory
output_dir = "dbfs:/mnt/silver"

# Write the transformed DataFrame to a single Parquet file in the specified directory
silver_df.write.mode("overwrite").option("header", "true").parquet(output_dir)



# Sentiment Column Code

In [0]:
from pyspark.sql.functions import col, sum

# Sum of all null values per column
null_counts = silver_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in silver_df.columns])

display(null_counts)

Video_id,Title,Published_at,Channel_id,Channel_title,Category_id,Tags,View_count,Likes,Dislikes,Comment_count,Thumbnail_link,Description
0,0,0,0,0,0,0,0,0,0,0,0,2


In [0]:
from pyspark.sql.functions import regexp_replace, col, lower, pandas_udf
from pyspark.sql.types import StringType
import pandas as pd

# Step-by-step data cleaning using regex replacements
# 1. Replace specific special characters (*, /, &) with an empty string
silver_df = silver_df.withColumn('Description', regexp_replace(col('Description'), r"[\*/&]", ""))

# 2. Remove all punctuation except word characters and whitespace
silver_df = silver_df.withColumn('Description', regexp_replace(col('Description'), r"[^\w\s]", ""))

# 3. Remove digits
silver_df = silver_df.withColumn('Description', regexp_replace(col('Description'), r"\d+", ""))

# 4. Replace multiple spaces with a single space
silver_df = silver_df.withColumn('Description', regexp_replace(col('Description'), r"\s{2,}", " "))

# 5. Convert text to lowercase
silver_df = silver_df.withColumn('Description', lower(col('Description')))

# 6. drop null/empty rows
silver_df = silver_df.filter(col("Description").isNotNull() & (col("Description") != ""))

# Display final result
display(silver_df)


Video_id,Title,Published_at,Channel_id,Channel_title,Category_id,Tags,View_count,Likes,Dislikes,Comment_count,Thumbnail_link,Description
U68MJz9DrI4,"Gucci Mane, Bruno Mars, Kodak Black - Wake Up in The Sky [Official Music Video]",2018-10-31,UCSugZEYrWbzqIWGD195V-YA,OfficialGucciMane,10,officialguccimane,673533,91342,1480,8204,https://i.ytimg.com/vi/U68MJz9DrI4/default.jpg,the official music video for gucci mane bruno mars and kodak blacks wake up in the sky available nowstreamdownload httpsguccimanelnktowakeupintheskyaydirector bruno mars florent dechardproduced by jeremy sullivansubscribe for more official content from gucci manehttpsatlanticlnktogmsubscribefollow gucci manehttpguccimaneonlinecomhttptwittercomguccihttpfacebookcomguccimanehttpinstagramcomlaflarefollow bruno marshttpwwwbrunomarscomhttpwwwinstagramcombrunomarshttpwwwtwittercombrunomarshttpwwwfacebookcombrunomarsfollow kodak blackhttpsofficialkodakblackcomhttpstwittercomkodakblackkhttpsfacebookcomtherealkodakblackhttpsinstagramcomkodakblackhttpssoundcloudcomkodakblackthe official youtube channel of atlantic records artist gucci mane subscribe for the latest music videos performances and moregucci kodak bruno guccimane brunomars kodakblack wakeupinthesky wuits musicvideo atlanticrecords atlantic
shWNgr0ifrs,Kevin Gates - Great Man [Official Music Video],2018-10-31,UCj2GTFekdV3EUsTVN8oaEqA,kevingatesTV,10,kevin gates,207289,18647,256,1707,https://i.ytimg.com/vi/shWNgr0ifrs/default.jpg,kevin gates great manstreamdownload httpskevingateslnktolucabrasisubscribe for more official content from kevin gateshttpskevingateslnktosubscribefollow gatestwitter httptwittercomkevin_gatesinstagram httpinstagramcomiamkevingatesfacebook httpsfacebookcomkvngatessoundcloud httpssoundcloudcomkevingateswebsite httpwwwkvngatescomthe official youtube channel of atlantic records artist kevin gates subscribe for the latest music videos performances and more
JIFMN986m8s,Worst Halloween Candy Taste Test (Day 3),2018-10-31,UC4PooiX37Pld1T8J5SYT-SQ,Good Mythical Morning,24,gmm,1054156,39129,1236,10153,https://i.ytimg.com/vi/JIFMN986m8s/default.jpg,weve arrived at the finals which of our ehh eight will be crowned the crappiest candy tune into ltat on saturday to see how the mythical crews brackets fared gmm watch todays gmmorehttpsyoutubelsmondrhkwant more gmm watch this season from the start httpbitlygmm_spick up official gmm and mythical merch athttpsmythicalstoreandhttpswwwamazoncommythicaldont miss our weekly podcast ear biscuitshttpsapplepodcastscomearbiscuitsfollow rhett linkinstagramhttpsinstagramcomrhettandlinkfacebookhttpsfacebookcomrhettandlinktwitterhttpstwittercomrhettandlinkwebsitehttpsmythicalcomcheck out our other mythical channelsgood mythical morehttpsyoutubecomgoodmythicalmorerhett linkhttpsyoutubecomrhettandlinkmythicalhttpsyoutubecomthisismythicalwant to send us somethinghttpsmythicalcomcontactsubmit your wheel of mythicality intro video herehttpsbitlygmmwheelintrointro animation by digital twigshttpswwwdigitaltwigscomintro outro music by jeff zeigler sarah schimeneckhttpswwwjeffzeiglercomsupplemental music from extreme production musichttpswwwextrememusiccommic the mouse by blue microphoneshttpswwwbluemiccommouse
0iy3HPxBFQY,James Corden & Ariana Grande Visit an Escape Room,2018-10-31,UCJ0uqCI0Vqr2Rrt1HseGirg,The Late Late Show with James Corden,24,The Late Late Show,896693,49252,289,2247,https://i.ytimg.com/vi/0iy3HPxBFQY/default.jpg,since ariana grande loves halloween and being scared james takes her to a haunted escape room and they both are terrified while trying to get out thank you to out escape rooms learn more about visiting one httpswwwoutcommore late late showsubscribe httpbitlycordenyoutubewatch full episodes httpbitlyenypwfacebook httponfbmepihlctwitter httpbitlyivqkinstagram httpbitlylatelategramwatch the late late show with james corden weeknights at am et pm ct only on cbsget new episodes of shows you love across devices the next day stream live tv and watch 

In [0]:
%pip install transformers torch

# Upgrade typing_extensions package to the latest version
%pip install --upgrade typing_extensions>=4.5

# Install the latest versions of transformers and torch
%pip install --upgrade transformers torch

%pip install -U pandas-profiling


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/18.2 MB ? eta -:--:--
   ━━━

In [0]:
#!pip install typing-extensions==4.7.1
#dbutils.library.restartPython()

In [0]:

from transformers import pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

# Load pre-trained sentiment analysis model from Hugging Face
sentiment_model = pipeline("sentiment-analysis")

# Define a function for sentiment classification
def classify_sentiment(text):
    if text:
        result = sentiment_model(text[:512])[0]  # Limiting to 512 characters for processing
        label = result['label']
        # Convert label to Positive, Neutral, or Negative
        if label == 'NEGATIVE':
            return 'Negative'
        elif label == 'POSITIVE':
            return 'Positive'
        else:
            return 'Neutral'
    return 'Neutral'  # Default for empty or null text

# Register the function as a UDF
sentiment_udf = udf(classify_sentiment, StringType())

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Apply Sentiment column to dataset

In [0]:
dbutils.fs.mount(
  source = "abfss://gold@endtoendstoresyn.dfs.core.windows.net/",
  mount_point = "/mnt/gold",
  extra_configs = configs)

True

In [0]:
#Apply Sentiment column to dataset
gold_df = silver_df.withColumn('Sentiment', sentiment_udf(col('Description')))

# Display the DataFrame to verify results
display(gold_df)


Video_id,Title,Published_at,Channel_id,Channel_title,Category_id,Tags,View_count,Likes,Dislikes,Comment_count,Thumbnail_link,Description,Sentiment
U68MJz9DrI4,"Gucci Mane, Bruno Mars, Kodak Black - Wake Up in The Sky [Official Music Video]",2018-10-31,UCSugZEYrWbzqIWGD195V-YA,OfficialGucciMane,10,officialguccimane,673533,91342,1480,8204,https://i.ytimg.com/vi/U68MJz9DrI4/default.jpg,the official music video for gucci mane bruno mars and kodak blacks wake up in the sky available nowstreamdownload httpsguccimanelnktowakeupintheskyaydirector bruno mars florent dechardproduced by jeremy sullivansubscribe for more official content from gucci manehttpsatlanticlnktogmsubscribefollow gucci manehttpguccimaneonlinecomhttptwittercomguccihttpfacebookcomguccimanehttpinstagramcomlaflarefollow bruno marshttpwwwbrunomarscomhttpwwwinstagramcombrunomarshttpwwwtwittercombrunomarshttpwwwfacebookcombrunomarsfollow kodak blackhttpsofficialkodakblackcomhttpstwittercomkodakblackkhttpsfacebookcomtherealkodakblackhttpsinstagramcomkodakblackhttpssoundcloudcomkodakblackthe official youtube channel of atlantic records artist gucci mane subscribe for the latest music videos performances and moregucci kodak bruno guccimane brunomars kodakblack wakeupinthesky wuits musicvideo atlanticrecords atlantic,Negative
shWNgr0ifrs,Kevin Gates - Great Man [Official Music Video],2018-10-31,UCj2GTFekdV3EUsTVN8oaEqA,kevingatesTV,10,kevin gates,207289,18647,256,1707,https://i.ytimg.com/vi/shWNgr0ifrs/default.jpg,kevin gates great manstreamdownload httpskevingateslnktolucabrasisubscribe for more official content from kevin gateshttpskevingateslnktosubscribefollow gatestwitter httptwittercomkevin_gatesinstagram httpinstagramcomiamkevingatesfacebook httpsfacebookcomkvngatessoundcloud httpssoundcloudcomkevingateswebsite httpwwwkvngatescomthe official youtube channel of atlantic records artist kevin gates subscribe for the latest music videos performances and more,Negative
JIFMN986m8s,Worst Halloween Candy Taste Test (Day 3),2018-10-31,UC4PooiX37Pld1T8J5SYT-SQ,Good Mythical Morning,24,gmm,1054156,39129,1236,10153,https://i.ytimg.com/vi/JIFMN986m8s/default.jpg,weve arrived at the finals which of our ehh eight will be crowned the crappiest candy tune into ltat on saturday to see how the mythical crews brackets fared gmm watch todays gmmorehttpsyoutubelsmondrhkwant more gmm watch this season from the start httpbitlygmm_spick up official gmm and mythical merch athttpsmythicalstoreandhttpswwwamazoncommythicaldont miss our weekly podcast ear biscuitshttpsapplepodcastscomearbiscuitsfollow rhett linkinstagramhttpsinstagramcomrhettandlinkfacebookhttpsfacebookcomrhettandlinktwitterhttpstwittercomrhettandlinkwebsitehttpsmythicalcomcheck out our other mythical channelsgood mythical morehttpsyoutubecomgoodmythicalmorerhett linkhttpsyoutubecomrhettandlinkmythicalhttpsyoutubecomthisismythicalwant to send us somethinghttpsmythicalcomcontactsubmit your wheel of mythicality intro video herehttpsbitlygmmwheelintrointro animation by digital twigshttpswwwdigitaltwigscomintro outro music by jeff zeigler sarah schimeneckhttpswwwjeffzeiglercomsupplemental music from extreme production musichttpswwwextrememusiccommic the mouse by blue microphoneshttpswwwbluemiccommouse,Negative
0iy3HPxBFQY,James Corden & Ariana Grande Visit an Escape Room,2018-10-31,UCJ0uqCI0Vqr2Rrt1HseGirg,The Late Late Show with James Corden,24,The Late Late Show,896693,49252,289,2247,https://i.ytimg.com/vi/0iy3HPxBFQY/default.jpg,since ariana grande loves halloween and being scared james takes her to a haunted escape room and they both are terrified while trying to get out thank you to out escape rooms learn more about visiting one httpswwwoutcommore late late showsubscribe httpbitlycordenyoutubewatch full episodes httpbitlyenypwfacebook httponfbmepihlctwitter httpbitlyivqkinstagram httpbitlylatelategramwatch the late late show with james corden weeknights at am et pm ct only on cbsget new episodes of shows you love across devices t

# Create Cleaned aggregated tables for final storage in synapse

In [0]:
from pyspark.sql.functions import round

# Aggregations view count
gold_df_avg_view_count = gold_df.groupBy("Sentiment").avg("View_count") \
    .withColumnRenamed("avg(View_count)", "Average_View_Count") \
    .withColumn("Average_View_Count", round("Average_View_Count")) 

display(gold_df_avg_view_count)

Sentiment,Average_View_Count
Positive,3377170.0
Negative,7174901.0


In [0]:
# Aggregation Likes
gold_df_avg_likes = gold_df.filter(gold_df.Sentiment.isin("Positive", "Negative")) \
    .groupBy("Sentiment") \
    .avg("Likes") \
    .withColumnRenamed("avg(Likes)", "Average_Likes") \
    .withColumn("Average_Likes", round("Average_Likes"))

display(gold_df_avg_likes)

Sentiment,Average_Likes
Positive,106761.0
Negative,229665.0


In [0]:
from pyspark.sql.functions import round

# Aggregations Dislikes
gold_df_avg_dislikes = gold_df.filter(gold_df.Sentiment.isin("Positive", "Negative")) \
    .groupBy("Sentiment") \
    .avg("Dislikes") \
    .withColumnRenamed("avg(Dislikes)", "Average_Dislikes") \
    .withColumn("Average_Dislikes", round("Average_Dislikes"))

display(gold_df_avg_dislikes)

Sentiment,Average_Dislikes
Positive,3931.0
Negative,7377.0


In [0]:
# Aggregation Comment Count
gold_df_avg_comment_count = gold_df.groupBy("Sentiment").avg("Comment_count") \
    .withColumnRenamed("avg(Comment_count)", "Average_Comment_Count") \
    .withColumn("Average_Comment_Count", round("Average_Comment_Count"))

display(gold_df_avg_comment_count)

Sentiment,Average_Comment_Count
Positive,12687.0
Negative,18016.0


In [0]:
# Save each table in its own directory within the mounted gold container in Parquet format
gold_df.write.mode("overwrite").parquet("/mnt/gold/gold_df")
gold_df_avg_view_count.write.mode("overwrite").parquet("/mnt/gold/gold_df_avg_view_count")
gold_df_avg_likes.write.mode("overwrite").parquet("/mnt/gold/gold_df_avg_likes")
gold_df_avg_dislikes.write.mode("overwrite").parquet("/mnt/gold/gold_df_avg_dislikes")
gold_df_avg_comment_count.write.mode("overwrite").parquet("/mnt/gold/gold_df_avg_comment_count")

In [0]:
# Unmount the containers to end the code
dbutils.fs.unmount("/mnt/bronze")
dbutils.fs.unmount("/mnt/silver")
dbutils.fs.unmount("/mnt/gold")

/mnt/bronze has been unmounted.
/mnt/silver has been unmounted.
/mnt/gold has been unmounted.


True